In [ ]:
!pip3 install torch

In [ ]:
!pip3 install transformers

     |████████████████████████████████| 2.1MB 4.5MB/s 
     |████████████████████████████████| 3.3MB 20.2MB/s 
     |████████████████████████████████| 901kB 34.6MB/s 


Достать эмбеддинг из pretrained:

In [ ]:
import torch
from transformers import (
    BertModel, BertConfig, BertTokenizer, TrainingArguments, 
    Trainer, LineByLineTextDataset, DataCollatorForLanguageModeling,
    DataCollatorWithPadding, DataCollator, BertForMaskedLM, BertForPreTraining
)

# logic from https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial
# this guy also has a video on youtube

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
config = BertConfig.from_pretrained('bert-base-multilingual-cased', output_hidden_states=True)
model = BertModel.from_pretrained('bert-base-multilingual-cased', config=config)

# how to train tokenizer?

text = 'hi there mother Russia'
# text = 'свободу всем нам'
encoded_input = tokenizer(text, return_tensors='pt')
outputs = model(**encoded_input)


# code from https://github.com/huggingface/transformers/issues/1827

print(len(outputs))  # 3

hidden_states = outputs[2]
print(len(hidden_states))  # 13

embedding_output = hidden_states[0]
attention_hidden_states = hidden_states[1:]


3
13


In [ ]:
'красиво' in tokenizer.vocab

False

In [ ]:
text = 'сильно очень любовь на всех'
def get_ids_tokens(text, tokenizer):
    tokens = tokenizer.tokenize('[CLS] ' + text + ' [SEP]')
    ids = tokenizer.encode(text)
    # assert tokens == tokenizer.convert_ids_to_tokens(ids)
    return ids, tokens

get_ids_tokens(text, tokenizer)

([101, 36341, 20598, 76976, 10122, 16989, 102],
 ['[CLS]', 'сильно', 'очень', 'любовь', 'на', 'всех', '[SEP]'])

In [ ]:
from scipy import spatial

def get_embeddings(text, tokenizer, model):
    encoded_input = tokenizer(text, return_tensors='pt')
    word_embeddings = model(**encoded_input)[2][-1][0].detach().numpy()[1:-1]
    return word_embeddings

In [ ]:
k, q, m, w = get_embeddings("king queen man woman", tokenizer, model)
spatial.distance.cosine(k - q, m - w)

0.9143929779529572

In [ ]:
last_four_sums = sum(hidden_states[-4:])

In [ ]:
# word embeddings четырёх токенов из входа
last_four_sums[0][1:][:-1]

tensor([[-3.1046,  0.4142,  1.6289,  ...,  2.0494,  2.3383, -0.8280],
        [ 1.0407,  0.7726,  0.1821,  ...,  2.3673,  3.1581, -0.0046],
        [-2.5376,  0.0475,  3.4065,  ...,  0.9091,  1.9999, -0.7310],
        [ 0.9056,  1.0325, -1.0536,  ...,  1.1029,  1.5878, -0.8780]],
       grad_fn=<SliceBackward>)

Дообучить на своей выборке

In [ ]:
# example from docs: https://colab.research.google.com/github/huggingface/blog/blob/master/notebooks/01_how_to_train.ipynb
# ориентир на доки: https://huggingface.co/transformers/model_doc/bert.html#bertformaskedlm, https://huggingface.co/transformers/main_classes/trainer.html#transformers.Trainer
#     https://huggingface.co/transformers/training.html#pytorch


In [ ]:
train_dataset = [
    'This is the first sentence.',
    'This is another one!',
    'This is the first sentence.',
    'This is another one!',
    'This is the first sentence.',
    'This is another one!'
] * 6

with open('train.txt', 'w') as fout:
    for s in train_dataset:
        print(s, file=fout)

# training from file

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path='train.txt',
    block_size=16,
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:124: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=True, mlm_probability=0.15)

In [ ]:
config = BertConfig.from_pretrained('bert-base-multilingual-cased', output_hidden_states=True)
model = BertForPreTraining.from_pretrained('bert-base-multilingual-cased', config=config)

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    # num_train_epochs=3,              # total # of training epochs
    # per_device_train_batch_size=16,  # batch size per device during training
    # per_device_eval_batch_size=64,   # batch size for evaluation
    # warmup_steps=500,                # number of warmup steps for learning rate scheduler
    # weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    data_collator=data_collator,
    # eval_dataset=test_dataset            # evaluation dataset
)

Some weights of BertForPreTraining were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
config = BertConfig.from_pretrained('bert-base-multilingual-cased', output_hidden_states=True)
model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased', config=config)

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    # num_train_epochs=3,              # total # of training epochs
    # per_device_train_batch_size=16,  # batch size per device during training
    # per_device_eval_batch_size=64,   # batch size for evaluation
    # warmup_steps=500,                # number of warmup steps for learning rate scheduler
    # weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    data_collator=data_collator,
    # eval_dataset=test_dataset            # evaluation dataset
)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=15, training_loss=1.7103668212890626, metrics={'train_runtime': 75.8802, 'train_samples_per_second': 0.198, 'total_flos': 922619927232.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 2003955712, 'train_mem_cpu_peaked_delta': 0})

In [ ]:
trainer.save_model('./myBERT')

Добываем эмбединги из зафайнтьюненной модели:

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
config = BertConfig.from_pretrained('./myBERT', output_hidden_states=True)
model = BertForMaskedLM.from_pretrained('./myBERT', config=config)

text = 'hi there mother Russia'
# text = 'свободу всем нам'
encoded_input = tokenizer(text, return_tensors='pt')
outputs = model(**encoded_input)

print(outputs.hidden_states)

(tensor([[[-0.3869,  0.1337, -0.1525,  ...,  0.2840, -0.4903, -0.0304],
         [-0.5909, -0.7388,  1.3123,  ..., -2.0042, -0.5036, -1.4133],
         [ 0.3446, -0.1882,  0.4469,  ...,  1.1253, -0.1963, -0.3069],
         [-0.0292, -0.4661,  1.6850,  ...,  1.0993, -0.2595, -1.1964],
         [ 0.4188,  0.4608, -0.7068,  ..., -0.1877,  1.4901, -0.8654],
         [ 0.7963,  0.5834,  1.0961,  ..., -0.1119,  0.9343, -0.0097]]],
       grad_fn=<NativeLayerNormBackward>), tensor([[[-0.0571, -0.0430, -0.0576,  ...,  0.0172, -0.2294,  0.1098],
         [-0.8273, -1.1358,  1.2048,  ..., -1.5437, -1.1892, -1.5865],
         [-0.0168,  0.3498,  0.3545,  ...,  0.9437, -0.2551, -0.4012],
         [-0.0937,  0.2248,  1.6361,  ...,  1.1256, -0.2115, -1.3953],
         [ 0.6026,  0.5909, -0.5920,  ..., -0.2991,  1.1359, -0.8510],
         [ 0.7826,  0.7687,  0.7355,  ...,  0.1072,  0.7755, -0.0773]]],
       grad_fn=<NativeLayerNormBackward>), tensor([[[-0.0499, -0.0518, -0.0187,  ..., -0.0170, -0.05

In [ ]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=T

In [ ]:
encoded_input

{'input_ids': tensor([[  101, 11520, 11155, 15293, 14664,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [ ]:
len(outputs.hidden_states)

13

In [ ]:
outputs.hidden_states[-1]

tensor([[[ 0.1040,  0.2657,  0.1623,  ...,  0.5423, -0.0681,  0.0268],
         [-0.1268, -0.0709,  0.2336,  ...,  0.4276, -0.1687, -0.0590],
         [ 0.4198,  0.2152,  0.1747,  ...,  0.5667,  0.0259,  0.1943],
         [-0.3378,  0.2825,  0.6063,  ..., -0.2832, -0.3945,  0.2580],
         [-0.4442, -0.1176,  0.4256,  ...,  0.4549, -0.3330, -0.4296],
         [-0.0604, -0.0766,  0.4610,  ...,  0.5612, -0.1893, -0.1336]]],
       grad_fn=<NativeLayerNormBackward>)

* Надо ещё обучить токенайзер (надо ли?)
* Надо доделать для разных m (дополнительный слой сверху?)
* Это для токенов, а не для слов.

Вопрос:

Some weights of BertForPreTraining were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

Нормально ли использовать MaskLM? Вместо PreTraining

# Кластеризация

In [ ]:
from scipy.spatial.distance import pdist, squareform#, euclidean
import numpy as np
from scipy.special import gamma
from collections import defaultdict
from itertools import product


WISHART_K = 4
WISHART_H = 0.2


def volume(r, m):
    return np.pi ** (m / 2) * r ** m / gamma(m / 2 + 1)


def significant(cluster, h, p):
    max_diff = max(abs(p[i] - p[j]) for i, j in product(cluster, cluster))

    # print(max_diff)
    return max_diff >= h


def partition(dist, l, r, order):
    if l == r:
        return l

    pivot = dist[order[(l + r) // 2]]
    left, right = l - 1, r + 1
    while True:
        while True:
            left += 1
            if dist[order[left]] >= pivot:
                break

        while True:
            right -= 1
            if dist[order[right]] <= pivot:
                break

        if left >= right:
            return right

        order[left], order[right] = order[right], order[left]

def nth_element(dist, order, k):
    l, r = 0, len(order) - 1
    while True:
        if l == r:
            break
        m = partition(dist, l, r, order)
        if m < k:
            l = m + 1
        elif m >= k:
            r = m

class WishartClusterization(object):
    def __init__(self, k=WISHART_K, h=WISHART_H):
        self.k = k
        self.h = h
        
    
    def fit(self, x):
        n = len(x)
        if isinstance(x[0], list):
            m = len(x[0])
        else:
            m = 1
        dist = squareform(pdist(x))

        dk = []
        for i in range(n):
            order = list(range(n))
            nth_element(dist[i], order, self.k - 1)
            dk.append(dist[i][order[self.k - 1]])

        # print(dk)

        p = [self.k / (volume(dk[i], m) * n) for i in range(n)]

        w = np.full(n, 0)
        completed = {0: False}
        last = 1
        vertices = set()
        for d, i in sorted(zip(dk, range(n))):
            neigh = set()
            neigh_w = set()
            clusters = defaultdict(list)
            for j in vertices:
                if dist[i][j] <= dk[i]:
                    neigh.add(j)
                    neigh_w.add(w[j])
                    clusters[w[j]].append(j)

            vertices.add(i)
            if len(neigh) == 0:
                w[i] = last
                completed[last] = False
                last += 1
            elif len(neigh_w) == 1:
                wj = next(iter(neigh_w))
                if completed[wj]:
                    w[i] = 0
                else:
                    w[i] = wj
            else:
                if all(completed[wj] for wj in neigh_w):
                    w[i] = 0
                    continue
                significant_clusters = set(wj for wj in neigh_w if significant(clusters[wj], self.h, p))
                if len(significant_clusters) > 1:
                    w[i] = 0
                    for wj in neigh_w:
                        if wj in significant_clusters:
                            completed[wj] = (wj != 0)
                        else:
                            for j in clusters[wj]:
                                w[j] = 0
                else:
                    if len(significant_clusters) == 0:
                        s = next(iter(neigh_w))
                    else:
                        s = next(iter(significant_clusters))
                    w[i] = s
                    for wj in neigh_w:
                        for j in clusters[wj]:
                            w[j] = s
        self.labels_ = w
        return self


In [ ]:
x = [[0], [0], [1], [10], [10], [11], [100], [101], [102]] * 2
x = last_four_sums[0][1:][:-1].detach()
w = WishartClusterization(k=WISHART_K, h=WISHART_H).fit(x)
w.labels_

array([1, 1, 1, 1])

In [ ]:
from google.colab import files
files.upload()

{}

In [ ]:
text = ''
with open('eng_text.txt', 'r') as file:
    for line in file:
        text += line
        print(1)
        if len(text.split()) > 1000:
            break

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [ ]:
get_ids_tokens(line, tokenizer)[1]

['[CLS]',
 'It',
 'was',
 'a',
 'bright',
 'cold',
 'day',
 'in',
 'April',
 ',',
 'and',
 'the',
 'clock',
 '##s',
 'were',
 'striking',
 'thirteen',
 '.',
 'Winston',
 'Smith',
 ',',
 'his',
 'chi',
 '##n',
 'nu',
 '##zzle',
 '##d',
 'into',
 'his',
 'breast',
 'in',
 'an',
 'effort',
 'to',
 'escape',
 'the',
 'vile',
 'wind',
 ',',
 'sl',
 '##ip',
 '##ped',
 'quickly',
 'through',
 'the',
 'glass',
 'doors',
 'of',
 'Victory',
 'Mansion',
 '##s',
 ',',
 'though',
 'not',
 'quickly',
 'enough',
 'to',
 'prevent',
 'a',
 's',
 '##wir',
 '##l',
 'of',
 'gr',
 '##itt',
 '##y',
 'dust',
 'from',
 'entering',
 'along',
 'with',
 'him',
 '.',
 '[SEP]']

In [ ]:
w = WishartClusterization(k=WISHART_K, h=WISHART_H).fit(get_embeddings(text, tokenizer, model))
w.labels_

RuntimeError: ignored

In [ ]:
get_ids_tokens(text, tokenizer)[1][-50:]
# get_embeddings(text, tokenizer, model)

['were',
 'just',
 'three',
 'other',
 'buildings',
 'of',
 'similar',
 'appearance',
 'and',
 'size',
 '.',
 'So',
 'completely',
 'did',
 'they',
 'dwa',
 '##rf',
 'the',
 'surrounding',
 'architecture',
 'that',
 'from',
 'the',
 'roof',
 'of',
 'Victory',
 'Mansion',
 '##s',
 'you',
 'could',
 'see',
 'all',
 'four',
 'of',
 'them',
 'simultaneously',
 '.',
 'The',
 'Ministry',
 'of',
 'Love',
 'was',
 'the',
 'really',
 'fri',
 '##ght',
 '##ening',
 'one',
 '.',
 '[SEP]']